# ===== 概要 =====
kaggleを始めたての人。これから機械学習を勉強したい人。基礎知識だけで実際に予測したことがない人向け。

ザックリデータを眺めるところからモデル作成までを解説します。

In [ ]:
import pandas as pd
import numpy as np

ライブラリをインポートします。

pandas:データを表に起こす

numpy:計算処理などに使う

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-feb-2021/train.csv")
test = pd.read_csv("../input/tabular-playground-series-feb-2021/test.csv")
sample = pd.read_csv("../input/tabular-playground-series-feb-2021/sample_submission.csv")

pandasでデータを読み込みます。

右端の|<と書いてある部分からデータを開き、"input"にあるcsvデータの右端"Copy file path"をクリックするとパスをコピーできます。

trainが学習データでtestが提出するためのデータ。

In [ ]:
print(sample.shape)
sample.head()

sampleは提出形式を確認するためのデータです。

idとtargetの２列にしてcsvファイルを提出しましょう。

In [ ]:
print(train.shape, test.shape)
train.head()

学習データのtrainは30万行です。

26列のデータがあり、targetが予測したいデータですね。

In [ ]:
pd.set_option("display.max_columns", 30)
train.head()

panasのset_optionを使えば途中に省略された列も表示できます。最大30列にしました。

catが文字のカテゴリデータ、contが数値データです。

In [ ]:
cat_cols = [c for c in train.columns if "cat" in c]
cat_cols

モデル作成のために前処理が必要なのでカテゴリデータの列を抽出しました。

[変数 for 変数 条件]でリストを作れます。

In [ ]:
num_cols = [c for c in train.columns if "cont" in c]
num_cols

数値データも同様。

# ===== データ確認 =====
ここからはデータの中身を見るだけなので、モデル作成だけを知りたい方は"モデル作成"まで飛んでください。

## ○カテゴリデータ

In [ ]:
train["cat0"].value_counts()

.value_count()で値ごとのデータ数を見れます。

"cat0"の列では"A"が281471行"B"が18529行あるみたいです。

In [ ]:
for col in cat_cols:
    print(col)
    print(train[col].value_counts())

for文を使って全カテゴリを確認しましょう。

列によってカテゴリ数が違います。

## ○数値データ

In [ ]:
import matplotlib.pyplot as plt
plt.hist(train["cont0"])
plt.show()

matplotlib.pyplotはグラフ描画のライブラリです。とりあえずインポートしてもいいくらい多用します。

.histでヒストグラムを作れるので"cont0"の分布を見ましょう。

外れ値だったり極端な偏りがあるのかザっと確認できます。

In [ ]:
_, ax = plt.subplots(2, 2, figsize = (16, 8))
ax[0][0].hist(train["cont0"])
ax[0][1].hist(train["cont1"])
ax[1][0].hist(train["cont2"])
ax[1][1].hist(train["cont3"])
plt.show()

.subplotsを使えば複数のグラフを一気に描画できます。

.subplots(行数, 列数)を設定してax[行][列]で描画する位置を指定します。

In [ ]:
for i in range(len(num_cols)):
    r = i // 5
    c = i % 5
    print(r, c, num_cols[i])

for文と.subplotsを組み合わせたいのでそれぞれの項目と行列番号を振り分けます。

In [ ]:
_, ax = plt.subplots(3, 5, figsize = (30, 12))
for i in range(len(num_cols)):
    r = i // 5
    c = i % 5
    ax[r][c].hist(train[num_cols[i]], bins = 100)
    ax[r][c].set_title(num_cols[i])
plt.show()

.subplotsで３行５列のグラフエリアを作成し、さっきのfor文の要領で行番号と列番号を振り分けました。

binsはヒストグラムの分割数を決める引数です。大きいほど細かな分布を確認できます。

例えば"cont1","cont4"はデータが離散してたり極端に偏っています。

In [ ]:
import seaborn as sns
_, ax = plt.subplots(figsize = (16, 6))
sns.boxplot(data = train[num_cols], ax = ax)
plt.show()

seabornもグラフ描画によく使われるライブラリです。よくsnsと略されます。

.boxplotはデータ分布や外れ値を簡単に確認するために便利です。

"cont1"～"cont13"にかけてだいたい0.2～0.8の間にデータが多く存在し、またスケールもほぼ同じだとわかります。

In [ ]:
plt.scatter(train["cont0"], train["target"], alpha = 0.3)
plt.show()

.scatterで散布図を描けます。

横軸は特徴量で縦軸はtargetにしました。

alphaを設定すると濃い部分にデータが偏っているとわかるので便利。（今回は意味を成してませんが...）

In [ ]:
_, ax = plt.subplots(3, 5, figsize = (30, 12))
for i in range(len(num_cols)):
    r = i // 5
    c = i % 5
    ax[r][c].scatter(train[num_cols[i]], train["target"], alpha = 0.3)
    ax[r][c].set_title(num_cols[i])
plt.show()

ヒストグラムの時と同じ要領で各項目とtargetとの散布図を描きました。

どれも四角に分布していてとても相関があるように見えませんね。

またヒストグラムでも確認しましたが"cont1"では謎の間隔があります。

"なんか意味ありげだなー"くらいに最初はとらえておきます。

In [ ]:
train[num_cols + ["target"]].head()

ここから相関係数を確認します。

まずは数値データとtargetのみを取り出すようにリストの結合(num_cols + ["target"])を使ってみましょう。

In [ ]:
corr = train[num_cols + ["target"]].corr()
corr

.corr()で相関係数が計算されます。

In [ ]:
corr.style.background_gradient(cmap = "bwr", axis = None, vmax = 1.0, vmin = -1.0)

.style.background_gradient()で表にカラーマップを施すことができます。

axis = Noneにしないと各列(行)方向に対してカラーリングするので気を付けましょう。

同じ値同士は当然1.0です。

targetと各数値データの相関係数を見るとどれも絶対値が0.1未満でほとんど相関していないとわかりますね。

In [ ]:
_, ax = plt.subplots(figsize = (8, 6))
sns.heatmap(corr, cmap = "bwr", vmax = 1.0, vmin = -1.0, ax = ax)
plt.show()

seabornの.heatmapでもヒートマップを作れます。

ほとんどtargetと相関がないとわかりますね。数値データは予測に貢献しないかもしれません。

また数値データ同士、例えば"cont5"と"cont12"の色が濃いつまり相関が高いように見えます。

In [ ]:
plt.scatter(train["cont5"], train["cont12"], alpha = 0.3)
plt.show()

相関が高そうな"cont5"と"cont12"を確認しました。

うーん...。

## ○総括
・カテゴリは2～14くらいに分かれている

・数値データはどれもスケールが似ている

・"cont1"は意味ありげな分布をしている

・数値データとtargetで相関がない（カテゴリデータが重要？主成分分析で見ると変わりそう？）

# ===== モデル作成 =====
LightGBMを作ります。

ランダムフォレストなどもっと初心者向けのモデルがありますが、作る労力はほぼ変わらないのでLightGBMの方がおススメです。

またXgboostを好む人もいます。

In [ ]:
train.head()

再度学習データを確認します。"cat0"～"cont13"までの列を使って"target"を予測します。

ここで大事なことは、カテゴリデータは文字のままではモデルに使えない点です。

なので例えば"A"を数値の１に置き換えるような変換をしましょう。

In [ ]:
for col in cat_cols:
    print(np.sort(train[col].unique()))
    print(np.sort(test[col].unique()))

trainもtestも変換が必要なのでそれぞれ片方にしかないカテゴリがないか確認します。

testにはあってtrainにないカテゴリがあると致命的ですが、trainにしかないのはセーフ。

In [ ]:
from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()
for col in cat_cols:
    train[col] = label.fit_transform(train[col])
    test[col] = label.transform(test[col])
train.head()

sklearnにLabelEncoderがあるので、これで文字を数値に変換していきます。

.fit_transformで例えば"A"を0に"B"を1に置き換える(fitする)作業と実際にデータを変換する(transformする)作業の両方を実行します。

testの方はすでにtrainでfitしているのでtransformだけでOKです。

In [ ]:
for col in cat_cols:
    print(np.sort(train[col].unique()))
    print(np.sort(test[col].unique()))

数値に変換されていますね。

In [ ]:
X = train.drop(columns = ["id", "target"])
y = train["target"]
print(X.shape, y.shape)
X.head()

学習に使う特徴量Xと予測したい目的変数yに分けます。

大文字X小文字yを使うのは人の好みですが、２次元データを大文字で１次元データを小文字で表す慣習があります。

In [ ]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits = 5)
for train_idx, valid_idx in kfold.split(X):
    print(len(train_idx), len(valid_idx))

実際に学習に使うデータ(train)とモデル性能を確認するためのデータ(valid)に分けます。

これをしないとモデルの良し悪しがわからないまま分析することになってしまいます。

分割方法はKFoldが一般的ですがコンペによっては違う分割をするので、ここは個人差（成績に差がつくところ）です。

今回は５回分割にしました。

KFoldはそれぞれ必ず１回は評価用データに回されるので５回分割すると全体の1/5が評価用データになります。つまり6万行です。

In [ ]:
for train_idx, valid_idx in kfold.split(X):
    X_train = X.iloc[train_idx, :]
    X_valid = X.iloc[valid_idx, :]
    y_train = y.iloc[train_idx]
    y_valid = y.iloc[valid_idx]
    print(X_train.shape, X_valid.shape, y_train.shape, y_valid.shape)

KFoldの分割をデータにあてはめます。

.splitでインデックス(行番号)として分割してくれるので、これを.iloc[行, 列]に使いましょう。

：は全てを意味するので、例えば.iloc[0, :]は０行目の全列です。

In [ ]:
import lightgbm as lgb

params = {
    "objective" : "regression",
    "metrics" : "rmse"
}

for train_idx, valid_idx in kfold.split(X):
    X_train = X.iloc[train_idx, :]
    X_valid = X.iloc[valid_idx, :]
    y_train = y.iloc[train_idx]
    y_valid = y.iloc[valid_idx]
    
    train_set = lgb.Dataset(X_train, y_train)
    valid_set = lgb.Dataset(X_valid, y_valid)
    
    model = lgb.train(
        params = params,
        train_set = train_set,
        valid_sets = [train_set, valid_set],
        num_boost_round = 300,
        early_stopping_rounds = 10,
        verbose_eval = 50
    )

モデルを作成します。

LightGBMに必要な行程は３つ。

①パラメータ設定

objectiveはモデルの種類を設定します。regressionは回帰です。分類だったらbinaryとかを使います。

metricsは評価指標(どうなったらモデルが良いのか)を決めます。今回は平均二乗誤差の平方根(rmse)です。

②.DatasetでLightGBM専用データに変換

.Dataset(特徴量, 目的変数)で変換します。

③.trainで学習

paramsは①のパラメータです。train_setは学習用データ。valid_setsは少なくとも評価用データだけは渡しましょう。

num_boost_roundは学習回数です。多ければ多いほど好ましいですがどこかで性能が頭打ちになるので多すぎると時間のムダです。

early_stopping_roundsはモデル性能(metrics)が指定した回数向上しなければ学習を止めます。今回は10にしました。

つまりnum_boost_roundを多めにしてearly_stopping_roundsを指定しておけばだいたい丸く収まります。

verbose_evalは何回目の学習ごとに結果出力するかを決めます。デフォルトは１ですが、出力結果が長すぎるので好みで変えましょう。


これで"モデルを作る"という目的までは達成できます。

ただしあくまでモデルを作っただけなので性能が不明です。

テニスで例えるなら、ボールを打つことができたけどコートに入ったのかフェンスに直撃したのかわからないといった感じ。

なので少し工夫を加えましょう。

In [ ]:
params = {
    "objective" : "regression",
    "metrics" : "rmse",
    "verbosity" : -1
}

models = []
OOF = np.zeros(y.shape[0])
results = []
for train_idx, valid_idx in kfold.split(X):
    X_train = X.iloc[train_idx, :]
    X_valid = X.iloc[valid_idx, :]
    y_train = y.iloc[train_idx]
    y_valid = y.iloc[valid_idx]
    
    train_set = lgb.Dataset(X_train, y_train)
    valid_set = lgb.Dataset(X_valid, y_valid)
    fold_results = {}
    model = lgb.train(
        params = params,
        train_set = train_set,
        valid_sets = [train_set, valid_set],
        num_boost_round = 300,
        early_stopping_rounds = 10,
        verbose_eval = 50,
        evals_result = fold_results
    )
    models.append(model)
    OOF[valid_idx] = model.predict(X_valid)
    results.append(fold_results)

modelsに各分割で作成したモデルを保存します。

OOFはOut Of Foldsの略で正解データyと予測値の誤差を確認するために用意しました。

resultsには学習過程を入れましょう。

paramsのverbosityを-1にすると余計な情報を出力しなくなります。

fold_resultsに各分割の学習過程を入れます。evals_resultに渡してください。

models.append()で作成したモデルをリストに追加します。

OOF[valid_idx]で評価用データのインデックスに予測結果を入れます。

In [ ]:
print(y.shape, OOF.shape)

OOFのサイズが実際のデータyと同じであると確認しました。

In [ ]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y, OOF)
rmse = np.sqrt(mse)
print(rmse)

mean_squared_errorで平均二乗誤差を計算できます。

yとOOFとの誤差を計算してmseにしましたが単位もスケールも二乗になるので平方根(rmse)をとりました。

rmseが0.845です。

In [ ]:
y.describe()

実際のデータyは標準偏差が0.887なのでボチボチの誤差です。

In [ ]:
results[0].keys()

resultsには学習過程を入れました。

辞書型になっていてキーは"training"と"valid_1"です。

In [ ]:
results[0]["training"]

"training"の中にはmetricsで指定した"rmse"が入っています。

In [ ]:
_, ax = plt.subplots(1, 5, figsize = (40, 6))
for i in range(len(results)):
    ax[i].plot(results[i]["training"]["rmse"], color = "red", label = "train")
    ax[i].plot(results[i]["valid_1"]["rmse"], color = "blue", label = "valid")
plt.legend(fontsize = 16)
plt.show()

この"rmse"を.plotでプロットしました。横軸が学習回数で縦軸が誤差(rmse)です。

赤線が学習用データ(train)で青線が評価用データ(valid)です。

trainの方が誤差が小さいですが、これは答えを知っているのであまり参考になりません。

validは100回目くらいで頭打ちになっていますね。

trainとvalidとの差が大きすぎると過学習しているので、今回も若干過学習気味です。

# ===== 提出 =====

In [ ]:
models[0]

modelsには学習モデルを保存しました。

このモデルを使って提出データを作ります。

In [ ]:
test.head()

testには予測したいtarget以外のデータが入っています。

In [ ]:
X_test = test.drop(columns = "id")
X_test.head()

"id"だけは予測に使わないので除去しました。

In [ ]:
pred = models[0].predict(X_test)
pred

models[0]で１個目のモデルが取り出せます。

.predictで予測結果を出しましょう。

In [ ]:
print(sample.shape, pred.shape)

予測結果のサイズがsampleのサイズと同じです。

In [ ]:
submit_preds = []
for model in models:
    pred = model.predict(X_test)
    submit_preds.append(pred)
print(len(submit_preds))

おなじように.predictを作成したモデル数分繰り返します。

その結果をsubmit_predsに入れました。

In [ ]:
np.mean(submit_preds, axis = 0).shape

５モデルそれぞれの予測結果をだしたので、その平均をとりましょう。

In [ ]:
np.mean(submit_preds, axis = 1).shape

axisは０か１で設定します。これは行方向か列方向かを設定します。

今回は１にすると"各モデルの予測結果の平均"となるので、全数が５になります。

さっきの０は"各予測結果の平均"になるので全数が200000です。

In [ ]:
sample

提出する形式を確認します。200000行2列です。

In [ ]:
test["id"]

"id"にはtestの"id"をそのまま使います。

In [ ]:
submit_preds = np.mean(submit_preds, axis = 0)

"target"には予測結果の平均をとりましょう。

In [ ]:
submit = pd.DataFrame()
submit["id"] = test["id"].copy()
submit["target"] = submit_preds
submit

pandasの.DataFrame()でカラのデータを作りました。

"id"と"target"のそれぞれにデータを入れましょう。

サイズもsampleと同じサイズです。

In [ ]:
submit.to_csv("submit.csv", index = False)

.to_csv()でcsvファイルを出力します。

実行すると右端|<をクリックして出てくる"output"にファイルが入っているはずです。

このファイルをダウンロードするか"Save Version"でノートブックを保存して提出しましょう。

# ===== 今後 =====
簡単なデータ確認からモデル作成までを解説しました。

後は特徴量を作ってみたりモデルのパラメータを変えたり別のモデルを試したりしてスコアを上げます。

ただしリーダーボード上のスコアが絶対ではなく、リーダーボードのデータに過剰フィットすると最終順位が決まるデータでのスコアが落ちます。

なのでこのノートで作ったOOFとの誤差が改善され、かつ、リーダーボードでの順位も上がるように調整したりします。